# Automated cheat detection from game player logs using Amazon SageMaker Autopilot

## Pre-requisites

In [ ]:
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import sagemaker
from sagemaker.predictor import csv_serializer

In [ ]:
boto3.Session(region_name='us-west-2').resource('s3').Bucket(bucket).download_file(os.path.join(prefix, 'sample_playerlog_for_training.csv'), 'sample_playerlog_for_training.csv')

In [ ]:
cheat = pd.read_csv('sample_playerlog_for_training.csv')
cheat[:10]

In [ ]:
boto3.Session(region_name='us-west-2').resource('s3').Bucket(bucket).download_file(os.path.join(prefix, 'sample_playerlog_for_validation.csv'), 'sample_playerlog_for_validation.csv')
validation_data = pd.read_csv('sample_playerlog_for_validation.csv')
validation_data[:10]

In [ ]:
prefix = 'sagemaker/cheat-detection/input'
sess   = sagemaker.Session()

uri = sess.upload_data(path="./sample_playerlog_for_training.csv", key_prefix=prefix)
print(uri)

## Evaluate model

In [ ]:
import boto3, sys

ep_name = 'cheat-detection-sample'
sm_rt = boto3.Session().client('runtime.sagemaker')

tn=tp=fn=fp=count=0

with open('sample_playerlog_for_validation.csv') as f:
    lines = f.readlines()
    for l in lines[1:2000]:  # Skip header
        l = l.split(',')      # Split CSV line into features
        label = l[8]          # Store 'yes'/'no' label
        del l[8]              # Remove label
        l = ','.join(l)       # Rebuild CSV line without label
                
        response = sm_rt.invoke_endpoint(EndpointName=ep_name, 
                                         ContentType='text/csv',       
                                         Accept='text/csv', Body=l)

        response = response['Body'].read().decode("utf-8")
        #print ("label %s response %s" %(label,response))

        if 'fraud' in label:
            # Sample is positive
            if 'fraud' in response:
                # True positive
                tp=tp+1
            else:
                # False negative
                fn=fn+1
        else:
            # Sample is negative
            if 'legit' in response:
                # True negative
                tn=tn+1
            else:
                # False positive
                fp=fp+1
        count = count+1
        if (count % 100 == 0):   
            sys.stdout.write(str(count)+' ')
            
print ("Done")

accuracy  = (tp+tn)/(tp+tn+fp+fn)
precision = tp/(tp+fp)
#recall    = tn/(tn+fn)
recall    = tp/(tp+fn)
f1        = (2*precision*recall)/(precision+recall)

print ("%.4f %.4f %.4f %.4f" % (accuracy, precision, recall, f1))
print ("tp:%d fn:%d tn:%d fp:%d" % (tp, fn, tn, fp))

In [ ]:
prefix = 'sagemaker/cheat-detection/sample-data'
sess   = sagemaker.Session()

uri = sess.upload_data(path="./sample_playerlog_for_validation.csv", key_prefix=prefix)
print(uri)

## Clean up

In [ ]:
sess.delete_endpoint(endpoint_name=ep_name)

In [ ]:
%%sh
aws s3 rm --recursive s3://sagemaker-us-east-2-ACCOUNT_NUMBER/sagemaker/cheat-detection/